See:
* http://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/
* https://medium.com/@rtjeannier/combining-data-sets-with-fuzzy-matching-17efcb510ab2

In [92]:
df1=pd.DataFrame({'A':['asdfg','ghjkl','qwert','cvbnvvc'],
                  'B':['AA','GGG','QQQQ','CC']})
df2=pd.DataFrame({'A_new':['gdjkl', 'asdgg', 'cdbnvvt', 'qwert','jhekr'],
                  'B_new':['GGG',   'AAA',   'CCC',     'QQQ',  'JJJ']})

In [93]:
df1

,A,B
0,asdfg,AA
1,ghjkl,GGG
2,qwert,QQQQ
3,cvbnvvc,CC


# Method 1:
https://stackoverflow.com/a/35406648/2268280

In [94]:
import pandas as pd
import fuzzywuzzy.process as fwp

In [95]:
fwp.extractOne(df1.loc[0,'A'],df2.A_new)

('asdgg', 80, 1)

In [103]:
df1['AA']=df1.A.apply(lambda s: list(fwp.extractOne(s,df2.A_new)))

In [104]:
df1['AA_merge']=df1['AA'].str[0]

In [112]:
df1=df1.merge(df2,left_on='AA_merge',right_on='A_new',how='left',suffixes=['','_df2'])#.drop('A_df2',axis='columns')

In [113]:
df1.B_new

0    AAA
1    GGG
2    QQQ
3    CCC
Name: B_new, dtype: object

In [118]:
df1

,A,B,AA,AA_merge,A_new,B_new
0,asdfg,AA,"[asdgg, 80, 1]",asdgg,asdgg,AAA
1,ghjkl,GGG,"[gdjkl, 80, 0]",gdjkl,gdjkl,GGG
2,qwert,QQQQ,"[qwert, 100, 3]",qwert,qwert,QQQ
3,cvbnvvc,CC,"[cdbnvvt, 71, 2]",cdbnvvt,cdbnvvt,CCC


In [116]:
import Levenshtein as lv

In [133]:
df1['lv']=df1.B.combine(df1.B_new,func=lambda x,y:int(lv.ratio(x,y)*100) )
df1.lv.combine(  
       df1.AA,lambda x,y: y[0] if y[1]>80 or (y[1]>70 and x>80) else None )

0     None
1    gdjkl
2    qwert
3     None
dtype: object

In [134]:
df1

,A,B,AA,AA_merge,A_new,B_new,lv
0,asdfg,AA,"[asdgg, 80, 1]",None,asdgg,AAA,80
1,ghjkl,GGG,"[gdjkl, 80, 0]",gdjkl,gdjkl,GGG,100
2,qwert,QQQQ,"[qwert, 100, 3]",qwert,qwert,QQQ,85
3,cvbnvvc,CC,"[cdbnvvt, 71, 2]",None,cdbnvvt,CCC,80
